In [1]:
#data wrangling tools
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np

#import models
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

#stats for ensembling

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

#Preprocessing
from sklearn.preprocessing import OrdinalEncoder

#Ensembling
from scipy import stats as st

In [2]:
def fold(fold_list,K):
    stack=np.column_stack((fold_list[i] for i in range(K)))
    mode=st.mode(stack,axis=1)
    return mode[0]

In [3]:
#Category encoder
def cat_encoder(X_train, X_test, cat_cols):
    encoder = OrdinalEncoder()
    train_encoder = encoder.fit_transform(X_train[cat_cols]).astype(int)
    test_encoder = encoder.transform(X_test[cat_cols]).astype(int)
    for col in cat_cols:
        X_train[col] = train_encoder[:, cat_cols.index(col)]
        X_test[col] = test_encoder[:, cat_cols.index(col)]
    encoder_cols = cat_cols
    return X_train, X_test, encoder_cols

In [4]:
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.feature_selection import VarianceThreshold

#Implementingthe Transformer class
class low_var(TransformerMixin):
    def __init__(self,threshold=0.3):
        self.threshold=threshold
    def fit(self,X,y=None):
        col_vars=X.var()
        self.col_to_drop=col_vars[col_vars<self.threshold].index
        return self
    def transform(self,X):
        assert self.col_to_drop is not None, 'Drop_col error, must be fitted before predict'
        X.drop(self.col_to_drop, axis=1, inplace=True)
        return X

In [5]:
#Reading the dataset
FILEPATH="/kaggle/input/Analytic-olympiad"
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

#Defining the features
target_col=df_train.columns.to_list()[-1]
cat_cols=df_train.select_dtypes(include="object").columns.to_list()
num_cols=df_train.select_dtypes(include=["int64","float64"]).columns.to_list()[:-2]

#Definging the training and testing dataset
X_train = df_train.drop([f'{target_col}'],axis=1).reset_index(drop=True)
y_train = df_train[f'{target_col}'].reset_index(drop=True)
X_test=df_test.copy()

In [6]:
# Drop_col
encoded_features = ['encoded_payment_' + str(i) for i in range(25)]
for col in encoded_features:
    X_train.drop([col], axis=1, inplace=True)
    X_test.drop([col],axis =1, inplace =True)
drop_cols = ['customer_id', 'firstname', 'lastname']
X_train.drop(drop_cols, axis=1, inplace=True)
X_test.drop(drop_cols, axis=1, inplace=True)

#Reassigning cat_cols
cat_cols=X_train.select_dtypes(include="object").columns.to_list()
num_cols=X_train.select_dtypes(include=["int64","float64"]).columns.to_list()[:-2]

#Removing columns with low variance
var_col=low_var(1)
X_train=pd.concat([var_col.fit_transform(X_train[num_cols]),X_train[cat_cols]],axis=1)
X_test=pd.concat([var_col.transform(X_test[num_cols]),X_test[cat_cols]],axis=1)

# category_encoders
X_train, X_test, cat_cols = cat_encoder(X_train, X_test, cat_cols)

In [7]:
class Splitter:
    def __init__(self, kfold=True, n_splits=5, cat_df=pd.DataFrame()):
        self.n_splits = n_splits
        self.kfold = kfold
        self.cat_df = cat_df

    def split_data(self, X, y, random_state_list):
        if self.kfold == 'skf':
            for random_state in random_state_list:
                kf = StratifiedKFold(n_splits=self.n_splits, random_state=random_state, shuffle=True)
                for train_index, val_index in kf.split(X, self.cat_df):
                    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
                    yield X_train, X_val, y_train, y_val
        elif self.kfold:
            for random_state in random_state_list:
                kf = KFold(n_splits=self.n_splits, random_state=random_state, shuffle=True)
                for train_index, val_index in kf.split(X, y):
                    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
                    yield X_train, X_val, y_train, y_val
        else:
            raise ValueError(f"Invalid kfold: Must be True")

In [8]:
class Classifier:
    def __init__(self):
        self.models = self._define_model()
        self.models_name = list(self._define_model().keys())
        self.len_models = len(self.models)
        
    def _define_model(self):
        lgbm_params={
        }
        
        xgb_params =  {
           
        }  
        
        rf_params =  {
            
        }
        
        cat_params = {

        }
        
       
        gdbt_params =  {
        }
        
        models = {
            'xgb': XGBClassifier(**xgb_params),
            'gdbt':GradientBoostingClassifier(**gdbt_params),
            'cat': CatBoostClassifier(**cat_params),
            'rf' : RandomForestClassifier(**rf_params),
            'lgbm':LGBMClassifier(**lgbm_params)
        }
        
        return models

In [9]:
kfold = True
n_splits = 5
random_state_list = [42] 
classifier = Classifier()
splitter = Splitter(kfold=kfold, n_splits=n_splits)
score_dict = dict(zip(classifier.models_name, [[] for _ in range(classifier.len_models)]))
test_predss = np.zeros((X_test.shape[0]))
final=[]
for i, (X_train_, X_val, y_train_, y_val) in enumerate(splitter.split_data(X_train, y_train, random_state_list=random_state_list)):
    clf=Classifier()
    models=clf.models
    test_preds = []
    oof_preds = []
    
    for name, model in models.items():
        model.fit(X_train_, y_train_)
        y_val_pred =model.predict(X_val).reshape(-1)
        test_pred = model.predict(X_test).reshape(-1)
        
        
        score= [roc_auc_score(y_val, y_val_pred)]
        score_dict[name].append(np.mean(score))
        
        oof_preds.append(y_val_pred)
        test_preds.append(test_pred)

    final.append(fold(test_preds,classifier.len_models))
        
test_predss=fold(final,n_splits)    

Learning rate set to 0.186788
0:	learn: 0.1394727	total: 276ms	remaining: 4m 35s
1:	learn: 0.0286887	total: 358ms	remaining: 2m 58s
2:	learn: 0.0084091	total: 436ms	remaining: 2m 24s
3:	learn: 0.0029898	total: 518ms	remaining: 2m 8s
4:	learn: 0.0013415	total: 593ms	remaining: 1m 58s
5:	learn: 0.0007231	total: 668ms	remaining: 1m 50s
6:	learn: 0.0004358	total: 746ms	remaining: 1m 45s
7:	learn: 0.0002980	total: 827ms	remaining: 1m 42s
8:	learn: 0.0001996	total: 918ms	remaining: 1m 41s
9:	learn: 0.0001745	total: 994ms	remaining: 1m 38s
10:	learn: 0.0001403	total: 1.07s	remaining: 1m 36s
11:	learn: 0.0001311	total: 1.15s	remaining: 1m 34s
12:	learn: 0.0001187	total: 1.22s	remaining: 1m 32s
13:	learn: 0.0001124	total: 1.29s	remaining: 1m 31s
14:	learn: 0.0001124	total: 1.36s	remaining: 1m 29s
15:	learn: 0.0001124	total: 1.42s	remaining: 1m 27s
16:	learn: 0.0001059	total: 1.5s	remaining: 1m 26s
17:	learn: 0.0001017	total: 1.57s	remaining: 1m 25s
18:	learn: 0.0001017	total: 1.64s	remaining: 1

162:	learn: 0.0000822	total: 11.2s	remaining: 57.6s
163:	learn: 0.0000822	total: 11.3s	remaining: 57.5s
164:	learn: 0.0000822	total: 11.3s	remaining: 57.4s
165:	learn: 0.0000822	total: 11.4s	remaining: 57.3s
166:	learn: 0.0000822	total: 11.5s	remaining: 57.3s
167:	learn: 0.0000822	total: 11.5s	remaining: 57.2s
168:	learn: 0.0000822	total: 11.6s	remaining: 57.1s
169:	learn: 0.0000822	total: 11.7s	remaining: 57s
170:	learn: 0.0000822	total: 11.8s	remaining: 57s
171:	learn: 0.0000822	total: 11.8s	remaining: 56.9s
172:	learn: 0.0000822	total: 11.9s	remaining: 56.8s
173:	learn: 0.0000822	total: 11.9s	remaining: 56.7s
174:	learn: 0.0000822	total: 12s	remaining: 56.6s
175:	learn: 0.0000822	total: 12.1s	remaining: 56.5s
176:	learn: 0.0000822	total: 12.1s	remaining: 56.5s
177:	learn: 0.0000822	total: 12.2s	remaining: 56.4s
178:	learn: 0.0000822	total: 12.3s	remaining: 56.3s
179:	learn: 0.0000822	total: 12.3s	remaining: 56.2s
180:	learn: 0.0000822	total: 12.4s	remaining: 56.1s
181:	learn: 0.0000

323:	learn: 0.0000822	total: 21.9s	remaining: 45.7s
324:	learn: 0.0000822	total: 22s	remaining: 45.7s
325:	learn: 0.0000822	total: 22.1s	remaining: 45.6s
326:	learn: 0.0000822	total: 22.1s	remaining: 45.5s
327:	learn: 0.0000822	total: 22.2s	remaining: 45.4s
328:	learn: 0.0000822	total: 22.2s	remaining: 45.4s
329:	learn: 0.0000822	total: 22.3s	remaining: 45.3s
330:	learn: 0.0000822	total: 22.4s	remaining: 45.2s
331:	learn: 0.0000822	total: 22.5s	remaining: 45.2s
332:	learn: 0.0000822	total: 22.5s	remaining: 45.1s
333:	learn: 0.0000822	total: 22.6s	remaining: 45s
334:	learn: 0.0000822	total: 22.7s	remaining: 45s
335:	learn: 0.0000822	total: 22.7s	remaining: 44.9s
336:	learn: 0.0000822	total: 22.8s	remaining: 44.8s
337:	learn: 0.0000822	total: 22.8s	remaining: 44.8s
338:	learn: 0.0000822	total: 22.9s	remaining: 44.7s
339:	learn: 0.0000822	total: 23s	remaining: 44.6s
340:	learn: 0.0000822	total: 23s	remaining: 44.5s
341:	learn: 0.0000822	total: 23.1s	remaining: 44.5s
342:	learn: 0.0000822	

482:	learn: 0.0000822	total: 32.5s	remaining: 34.8s
483:	learn: 0.0000822	total: 32.6s	remaining: 34.7s
484:	learn: 0.0000822	total: 32.6s	remaining: 34.7s
485:	learn: 0.0000822	total: 32.7s	remaining: 34.6s
486:	learn: 0.0000822	total: 32.8s	remaining: 34.5s
487:	learn: 0.0000822	total: 32.8s	remaining: 34.5s
488:	learn: 0.0000822	total: 32.9s	remaining: 34.4s
489:	learn: 0.0000822	total: 33s	remaining: 34.3s
490:	learn: 0.0000822	total: 33s	remaining: 34.3s
491:	learn: 0.0000822	total: 33.1s	remaining: 34.2s
492:	learn: 0.0000822	total: 33.2s	remaining: 34.1s
493:	learn: 0.0000822	total: 33.2s	remaining: 34.1s
494:	learn: 0.0000822	total: 33.3s	remaining: 34s
495:	learn: 0.0000822	total: 33.4s	remaining: 33.9s
496:	learn: 0.0000822	total: 33.4s	remaining: 33.8s
497:	learn: 0.0000822	total: 33.5s	remaining: 33.8s
498:	learn: 0.0000822	total: 33.6s	remaining: 33.7s
499:	learn: 0.0000822	total: 33.6s	remaining: 33.6s
500:	learn: 0.0000822	total: 33.7s	remaining: 33.6s
501:	learn: 0.0000

643:	learn: 0.0000822	total: 43.3s	remaining: 24s
644:	learn: 0.0000822	total: 43.4s	remaining: 23.9s
645:	learn: 0.0000822	total: 43.5s	remaining: 23.8s
646:	learn: 0.0000822	total: 43.5s	remaining: 23.8s
647:	learn: 0.0000822	total: 43.6s	remaining: 23.7s
648:	learn: 0.0000822	total: 43.7s	remaining: 23.6s
649:	learn: 0.0000822	total: 43.8s	remaining: 23.6s
650:	learn: 0.0000822	total: 43.8s	remaining: 23.5s
651:	learn: 0.0000822	total: 43.9s	remaining: 23.4s
652:	learn: 0.0000822	total: 44s	remaining: 23.4s
653:	learn: 0.0000822	total: 44s	remaining: 23.3s
654:	learn: 0.0000822	total: 44.1s	remaining: 23.2s
655:	learn: 0.0000822	total: 44.2s	remaining: 23.2s
656:	learn: 0.0000822	total: 44.2s	remaining: 23.1s
657:	learn: 0.0000822	total: 44.3s	remaining: 23s
658:	learn: 0.0000822	total: 44.4s	remaining: 22.9s
659:	learn: 0.0000822	total: 44.4s	remaining: 22.9s
660:	learn: 0.0000822	total: 44.5s	remaining: 22.8s
661:	learn: 0.0000822	total: 44.6s	remaining: 22.7s
662:	learn: 0.000082

...

464:	learn: 0.0000758	total: 2m 6s	remaining: 2m 25s
465:	learn: 0.0000758	total: 2m 7s	remaining: 2m 25s
466:	learn: 0.0000758	total: 2m 7s	remaining: 2m 25s
467:	learn: 0.0000758	total: 2m 7s	remaining: 2m 25s
468:	learn: 0.0000758	total: 2m 8s	remaining: 2m 25s
469:	learn: 0.0000758	total: 2m 8s	remaining: 2m 24s
470:	learn: 0.0000758	total: 2m 8s	remaining: 2m 24s
471:	learn: 0.0000758	total: 2m 8s	remaining: 2m 23s
472:	learn: 0.0000758	total: 2m 8s	remaining: 2m 23s
473:	learn: 0.0000758	total: 2m 8s	remaining: 2m 22s
474:	learn: 0.0000758	total: 2m 8s	remaining: 2m 22s
475:	learn: 0.0000758	total: 2m 8s	remaining: 2m 21s
476:	learn: 0.0000758	total: 2m 8s	remaining: 2m 21s
477:	learn: 0.0000758	total: 2m 9s	remaining: 2m 21s
478:	learn: 0.0000758	total: 2m 9s	remaining: 2m 20s
479:	learn: 0.0000758	total: 2m 9s	remaining: 2m 20s
480:	learn: 0.0000758	total: 2m 9s	remaining: 2m 19s
481:	learn: 0.0000758	total: 2m 9s	remaining: 2m 19s
482:	learn: 0.0000758	total: 2m 9s	remaining: 

618:	learn: 0.0000758	total: 2m 51s	remaining: 1m 45s
619:	learn: 0.0000758	total: 2m 51s	remaining: 1m 45s
620:	learn: 0.0000758	total: 2m 51s	remaining: 1m 44s
621:	learn: 0.0000758	total: 2m 51s	remaining: 1m 44s
622:	learn: 0.0000758	total: 2m 52s	remaining: 1m 44s
623:	learn: 0.0000758	total: 2m 52s	remaining: 1m 43s
624:	learn: 0.0000758	total: 2m 52s	remaining: 1m 43s
625:	learn: 0.0000758	total: 2m 53s	remaining: 1m 43s
626:	learn: 0.0000758	total: 2m 53s	remaining: 1m 43s
627:	learn: 0.0000758	total: 2m 53s	remaining: 1m 42s
628:	learn: 0.0000758	total: 2m 53s	remaining: 1m 42s
629:	learn: 0.0000758	total: 2m 54s	remaining: 1m 42s
630:	learn: 0.0000758	total: 2m 54s	remaining: 1m 42s
631:	learn: 0.0000758	total: 2m 54s	remaining: 1m 41s
632:	learn: 0.0000758	total: 2m 55s	remaining: 1m 41s
633:	learn: 0.0000758	total: 2m 55s	remaining: 1m 41s
634:	learn: 0.0000758	total: 2m 55s	remaining: 1m 41s
635:	learn: 0.0000758	total: 2m 56s	remaining: 1m 40s
636:	learn: 0.0000758	total:

771:	learn: 0.0000758	total: 3m 37s	remaining: 1m 4s
772:	learn: 0.0000758	total: 3m 37s	remaining: 1m 3s
773:	learn: 0.0000758	total: 3m 37s	remaining: 1m 3s
774:	learn: 0.0000758	total: 3m 37s	remaining: 1m 3s
775:	learn: 0.0000758	total: 3m 37s	remaining: 1m 2s
776:	learn: 0.0000758	total: 3m 38s	remaining: 1m 2s
777:	learn: 0.0000758	total: 3m 38s	remaining: 1m 2s
778:	learn: 0.0000758	total: 3m 38s	remaining: 1m 1s
779:	learn: 0.0000758	total: 3m 38s	remaining: 1m 1s
780:	learn: 0.0000758	total: 3m 38s	remaining: 1m 1s
781:	learn: 0.0000758	total: 3m 38s	remaining: 1m
782:	learn: 0.0000758	total: 3m 38s	remaining: 1m
783:	learn: 0.0000758	total: 3m 38s	remaining: 1m
784:	learn: 0.0000758	total: 3m 38s	remaining: 59.9s
785:	learn: 0.0000758	total: 3m 38s	remaining: 59.6s
786:	learn: 0.0000758	total: 3m 39s	remaining: 59.3s
787:	learn: 0.0000758	total: 3m 39s	remaining: 59s
788:	learn: 0.0000758	total: 3m 39s	remaining: 58.6s
789:	learn: 0.0000758	total: 3m 39s	remaining: 58.3s
790:

928:	learn: 0.0000758	total: 4m 14s	remaining: 19.4s
929:	learn: 0.0000758	total: 4m 14s	remaining: 19.2s
930:	learn: 0.0000758	total: 4m 14s	remaining: 18.9s
931:	learn: 0.0000758	total: 4m 14s	remaining: 18.6s
932:	learn: 0.0000758	total: 4m 14s	remaining: 18.3s
933:	learn: 0.0000758	total: 4m 15s	remaining: 18s
934:	learn: 0.0000758	total: 4m 15s	remaining: 17.7s
935:	learn: 0.0000758	total: 4m 15s	remaining: 17.4s
936:	learn: 0.0000758	total: 4m 15s	remaining: 17.2s
937:	learn: 0.0000758	total: 4m 15s	remaining: 16.9s
938:	learn: 0.0000758	total: 4m 15s	remaining: 16.6s
939:	learn: 0.0000758	total: 4m 15s	remaining: 16.3s
940:	learn: 0.0000758	total: 4m 15s	remaining: 16s
941:	learn: 0.0000758	total: 4m 15s	remaining: 15.7s
942:	learn: 0.0000758	total: 4m 15s	remaining: 15.5s
943:	learn: 0.0000758	total: 4m 15s	remaining: 15.2s
944:	learn: 0.0000758	total: 4m 16s	remaining: 14.9s
945:	learn: 0.0000758	total: 4m 16s	remaining: 14.6s
946:	learn: 0.0000758	total: 4m 16s	remaining: 14.

C:\Users\VaishakhRaveendran\AppData\Local\Temp\ipykernel_12840\1986525794.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  stack=np.column_stack((fold_list[i] for i in range(K)))


In [15]:
result=pd.read_csv('submission_trivial_base.csv')
result['final_close_flag']=test_predss

In [16]:
result.to_csv('submission_XXFFF.csv',index=False)